# Results 
- Plot training/test telemetry values, predictions, smoothed errors, and predicted and actual anomalies
- A specified results file from the `results` dir is used to highlight anomalous regions and read in data from the `data` dir

In [1]:
# Imports

# Update paths for custom modules
import os
import sys
import numpy as np
import pandas as pd
import cufflinks as cf

sys.path.insert(0, '/home/alexey/School/Research/submodules')
sys.path.insert(0, '/home/alexey/School/Research/submodules/telemanom')
from telemanom.plotting import Plotter
from mypkg.src import *

from plotly.offline import download_plotlyjs, init_notebook_mode
cf.go_offline()
init_notebook_mode(connected=True)

%load_ext autoreload
%autoreload 2

## Examine labeled_anomalies.csv

In [2]:
def examine_labels():
    # Show labeled_anomalies.csv file
    with pd.option_context('display.max_rows', 90, 'display.max_columns', 40):
        la_file = "/home/alexey/School/Research/submodules/telemanom/labeled_anomalies.csv"
        values = pd.read_csv(la_file)
        # Show full table
        display(values)

if True:
    examine_labels()
    print()
    print_data.print_sc_anomaly_stats('MSL')
    print()
    print_data.print_sc_anomaly_stats('SMAP')

,chan_id,spacecraft,anomaly_sequences,class,num_values
0,P-1,SMAP,"[[2149, 2349], [4536, 4844], [3539, 3779]]","[contextual, contextual, contextual]",8505
1,S-1,SMAP,"[[5300, 5747]]",[point],7331
2,E-1,SMAP,"[[5000, 5030], [5610, 6086]]","[contextual, contextual]",8516
3,E-2,SMAP,"[[5598, 6995]]",[point],8532
4,E-3,SMAP,"[[5094, 8306]]",[point],8307
5,E-4,SMAP,"[[5450, 8261]]",[point],8354
6,E-5,SMAP,"[[5600, 5920]]",[point],8294
7,E-6,SMAP,"[[5610, 5675]]",[point],8300
8,E-7,SMAP,"[[5394, 5674]]",[point],8310
9,E-8,SMAP,"[[5400, 6022]]",[point],8532



msl:
       anomaly_sequences
count          27.000000
mean            1.333333
std             0.554700
min             1.000000
25%             1.000000
50%             1.000000
75%             2.000000
max             3.000000
Total anomalies: 36

smap:
       anomaly_sequences
count          55.000000
mean            1.254545
std             0.584307
min             1.000000
25%             1.000000
50%             1.000000
75%             1.000000
max             3.000000
Total anomalies: 69


## Check what TranAD is plotting after testing

In [3]:
def analyze_smap_labels_tranad():
    fn = "P-1"
    fn_labels = np.load(f"/home/alexey/School/Research/submodules/TranAD/processed/SMAP/{fn}_labels.npy")
    true_labels = np.load(f"/home/alexey/School/Research/Datasets/Preprocessed/SMAP/Both/Labels/{fn}.npy")

    print_data.print_chan_split_shape('P-1', 'Train')
    print_data.print_chan_split_shape('P-1', 'Test')
    print_data.print_chan_split_shape('P-1', 'Both')

    print(true_labels)

    print(fn_labels[2148:2150, 0])
    print(fn_labels[2348:2350, 0])

    print(fn_labels[4535:4537, 0])
    print(fn_labels[4843:4845, 0])

    print(fn_labels[3538:3540, 0])
    print(fn_labels[3778:3780, 0])

if True:
    analyze_smap_labels_tranad()

Shape of channel "P-1" split "Train" is (2872, 25)
Shape of channel "P-1" split "Test" is (8505, 25)
Shape of channel "P-1" split "Both" is (11377, 25)
[[5021 5221]
 [7408 7716]
 [6411 6651]]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]


## Print Precision and Recall

In [4]:
if True:
    print("MSL:")
    print_data.print_prec_rec_sc('MSL')
    print("SMAP:")
    print_data.print_prec_rec_sc('SMAP')

MSL:
    Precision: 0.96
    Recall: 0.69
    F0.5: 0.89
    F1.0: 0.81
SMAP:
    Precision: 0.84
    Recall: 0.90
    F0.5: 0.85
    F1.0: 0.87


## Print SMAP and MSL streams

In [5]:

if True:
    chans = helpers.get_all_chan_id_from_sc('SMAP')
    print(f"SMAP: {len(chans)}")
    print(chans)

if True:
    chans = helpers.get_all_chan_id_from_sc('MSL')
    print(f"MSL: {len(chans)}")
    print(chans)

if True:
    chan_id = "A-1"
    sc = helpers.get_sc_from_chan_id(chan_id)
    print(f"Channel {chan_id} is {sc}")
    

SMAP: 55
['P-1', 'S-1', 'E-1', 'E-2', 'E-3', 'E-4', 'E-5', 'E-6', 'E-7', 'E-8', 'E-9', 'E-10', 'E-11', 'E-12', 'E-13', 'A-1', 'D-1', 'P-2', 'P-3', 'D-2', 'D-3', 'D-4', 'A-2', 'A-3', 'A-4', 'G-1', 'G-2', 'D-5', 'D-6', 'D-7', 'F-1', 'P-4', 'G-3', 'T-1', 'T-2', 'D-8', 'D-9', 'F-2', 'G-4', 'T-3', 'D-11', 'D-12', 'B-1', 'G-6', 'G-7', 'P-7', 'R-1', 'A-5', 'A-6', 'A-7', 'D-13', 'P-2', 'A-8', 'A-9', 'F-3']
MSL: 27
['M-6', 'M-1', 'M-2', 'S-2', 'P-10', 'T-4', 'T-5', 'F-7', 'M-3', 'M-4', 'M-5', 'P-15', 'C-1', 'C-2', 'T-12', 'T-13', 'F-4', 'F-5', 'D-14', 'T-9', 'P-14', 'T-8', 'P-11', 'D-15', 'D-16', 'M-7', 'F-8']
Channel A-1 is SMAP


## Summary of parameters and results

In [6]:
#run_ids = ["2018-05-19_15.00.10", "yes_avg", "no_avg"]
#run_ids = ["yes_avg", "no_avg"]
run_ids = ["2018-05-19_15.00.10"]

if True:
    print("Run-time parameters:")
    for run_id in run_ids:
        print(f"{run_id}:\n")
        print_data.print_run_params(run_id)
        print()

if True:
    print("Results summary:")
    for run_id in run_ids:
        print(run_id)
        print_data.print_run_tp_fp_fn(run_id)
        print()
        print_data.print_run_prec_rec(run_id)
        print()


Run-time parameters:
2018-05-19_15.00.10:

batch_size: 70
dropout: 0.3
epochs: 35
error_buffer: 100
l_s: 250
layers: [80, 80]
loss_metric: mse
lstm_batch_size: 64
min_delta: 0.0003
n_predictions: 10
optimizer: adam
p: 0.13
patience: 10
predict: False
smoothing_perc: 0.05
train: False
validation_split: 0.2
window_size: 30

Results summary:
2018-05-19_15.00.10
    True Positives: 87
    False Positives: 13
    False Negatives: 18

Total:
    Precision: 0.87
    Recall: 0.83
    F0.5: 0.86
    F1.0: 0.85



## Interactive inline Plotly charts for viewing `y_test`, `y_hat`, and `smoothed errors (e_s)`
- **Blue** highlighted regions indicate anomalous sequences detected by the system
- If available, **Red** highlighted regions indicate true anomalous regions
- Can also optionally plot training data by setting `plot_train=True`

In [7]:

plot_params = [0]
plot_data.plot_channel_split_params('P-1', plot_params=plot_params, split='Both')
plot_data.plot_channel_split_params('P-1', plot_params=plot_params, split='Test')


100%|██████████| 53/53 [00:00<00:00, 1148.35it/s]


100%|██████████| 53/53 [00:00<00:00, 2265.20it/s]

In [8]:
run_ids = ["2018-05-19_15.00.10"]
for i in range(len(run_ids)):
    channel='P-1'
    print_data.print_chan_split_shape(channel, 'Test')
    print_data.print_chan_split_shape(channel, 'Train')
    print_data.print_chan_split_shape(channel, 'Both')
    print(run_ids[i])
    plotter = Plotter(run_ids[i])
    #plotter.channel_result_summary(channel)
    plotter.plot_channel(channel, plot_errors=True, plot_both=True, plot_test=False, plot_train=False)
    print()

Shape of channel "P-1" split "Test" is (8505, 25)
Shape of channel "P-1" split "Train" is (2872, 25)
Shape of channel "P-1" split "Both" is (11377, 25)
2018-05-19_15.00.10
Train shape 25.24%: (2872, 25)
Test shape 74.76%: (8505, 25)
Spacecraft: SMAP
Channel: P-1
Normalized prediction error: 0.09
Anomaly class(es): [contextual, contextual, contextual]
------------------
True Positives: 3
False Positives: 1
False Negatives: 0
------------------


,run_id,chan_id,num_train_values,num_test_values,n_predicted_anoms,normalized_pred_error,anom_scores,false_positives,false_negatives,true_positives,fp_sequences,tp_sequences,num_true_anoms,scores,spacecraft,anomaly_sequences,class
0,2018-05-19_15.00.10,P-1,2612,8245,4,0.085043,"[2.0284713382242288, 2.163556600069587, 2.2936...",1,0,3,"[[3190, 3329]]","[(2130, 2349), (3540, 3679), (4520, 4589)]",3,"[2.0284713382242288, 2.163556600069587, 2.2936...",SMAP,"[[2149, 2349], [4536, 4844], [3539, 3779]]","[contextual, contextual, contextual]"


Predicted anomaly scores: [2.0284713382242288, 2.163556600069587, 2.2936227645004643, 0.9175092092214928]
Number of values: 8505


## Print Percentage of Track as Train/Test Data

In [9]:
print_data.print_dataset_split_stats(sc='MSL_SMAP')

msl_smap training percentages:
        Training
count  80.000000
mean   30.225658
std    11.465935
min     3.791009
25%    25.011488
50%    25.288671
75%    33.303258
max    58.333333

msl_smap testing percentages:
         Testing
count  80.000000
mean   69.774342
std    11.465935
min    41.666667
25%    66.696742
50%    74.711329
75%    74.988512
max    96.208991



## Get Channel Result Information

In [10]:

run_id = run_ids[0]
channel_id = 'T-5'
result_df = pd.read_csv(os.path.join('/home/alexey/School/Research/submodules/telemanom', 'results', '{}.csv'.format(run_id)))
display(result_df[result_df['chan_id'] == channel_id])


,run_id,chan_id,num_train_values,num_test_values,n_predicted_anoms,normalized_pred_error,anom_scores,false_positives,false_negatives,true_positives,fp_sequences,tp_sequences,num_true_anoms,scores,spacecraft,anomaly_sequences,class
61,2018-05-19_15.00.10,T-5,2012,1958,1,0.00788,[12.319131674587176],0,0,1,[],"[(1114, 1381)]",1,[12.319131674587176],MSL,"[[1200, 1225]]",[point]


## Parse data

In [11]:
if False:
    parse_data.parse_tracks('SMAP')
    parse_data.parse_tracks('MSL')


## Print nan info

In [12]:
print("MSL:")
print_data.print_dataset_nan_info('MSL')
print()
print("SMAP:")
print_data.print_dataset_nan_info('SMAP')


MSL:
Parameters without any nans (55):
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]
Parameters with some nans (0):
[]
Parameters with all nans (0):
[]
Byte parameters (0):
[]

SMAP:
Parameters without any nans (25):
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Parameters with some nans (0):
[]
Parameters with all nans (0):
[]
Byte parameters (0):
[]


In [13]:
print("MSL")
print_data.print_dataset_length_stats('MSL')
print()
print("SMAP")
print_data.print_dataset_length_stats('SMAP')
print_data.print_dataset_length_stats('MSL_SMAP')

MSL
                  0
count     27.000000
mean    4890.592593
std     2035.411795
min     1535.000000
25%     3628.000000
50%     4422.000000
75%     6064.500000
max    10408.000000
Total data points: 132046

SMAP
                  0
count     53.000000
mean   10629.283019
std     1309.380599
min     5135.000000
25%    10225.000000
50%    11180.000000
75%    11396.000000
max    11521.000000
Total data points: 563352
                  0
count     80.000000
mean    8692.475000
std     3154.160543
min     1535.000000
25%     5599.000000
50%    10308.500000
75%    11316.500000
max    11521.000000
Total data points: 695398
